In [ ]:
import os
import sys
import pandas as pd

sys.path.insert(0, '../')
from utils import (base_path, process_fwf_data, process_tabular_data,
                   concat_data_add_source, find_duplicate_cols, create_county_redshift_table)

In [ ]:
county_name = 'Alameda'

# I. Initial data processing 

**Start:** file(s) in format given by county assessor

**End:** csv file for each data source

In [ ]:
data_dir = os.path.join(base_path, 'Alameda County Assessor Data')

##  Initialize column widths dictionary

Process:

- Copied columns from File Layout for Job IE673 - Property Characteristics.doc into excel file
- copied the column name and width columns into Sublime
- did text replace (replace tab with "': ", replace newline with ", '")

In [ ]:
fname = 'IE673 - 2018-2019'

d = {"Assessor's Parcel Number (APN) sort format": 13,
     'APN print format': 15,
     'Primary tax rate area (TRA)': 2,
     'Secondary TRA': 3,
     'Situs street number': 10,
     'Situs street name': 50,
     'Situs unit number': 10,
     'Situs city': 30,
     'Situs zip code': 5,
     'Situs zip+4': 4,
     'Use code': 4,
     'Property characteristic change date (CCYYMMDD)': 8,
     'Number of units': 5,
     'Building class': 4,
     'Building effective year': 4,
     'Building area': 7,
     'Lot size': 8,
     'Number of buildings': 3,
     'Number of stories': 3,
     'Number of rooms': 3,
     'Number of bedrooms': 3,
     'Land/improvement ratio': 3,
     'Percent office': 3,
     "Acre code - 'A' if lot size is in acres, blank otherwise": 1,
     'Land flag': 1,
     'Conformity flag': 1,
     'Additions area': 7,
     'Misc area': 7,
     'Rentable space': 7,
     'Year built': 4,
     'Number of bathrooms': 5,
     'Condo type': 1,
     'Elevator flag': 1,
     'Swimming pool code': 1,
     'Swimming pool year built': 4,
     'Remodel code': 3,
     'Condition code': 3,
     'Parking code': 4,
     'Amenities code': 4,
     'Unit floor': 2,
     'Wall height': 3,
     'View code': 1,
     'Slope code': 1,
     'Topography code': 1,
     'Hazards code': 2,
     'Economic unit flag': 1,
     'End date (CCYYMMDD) Date APN was inactivated -': 8}

fname_d = process_fwf_data(data_folder=data_dir, fname=fname,
                           colwidths_dict=d, county_name=county_name)
# reload to check conversion
df = pd.read_csv(fname_d['df_path'])
df.head()

## Taxroll data

In [ ]:
# due to ProgrammingError: column "situs_city" duplicated error thrown by create_county_redshift_table(county_name),
# had to rename these columns:
# ['situs city', 'situs street name', 'situs street number', 'use code']

column_rename_d = {'Situs City': 'situs city taxroll',
                   'Situs Street Name': 'situs street name taxroll',
                   'Situs Street Number': 'situs street number taxroll',
                   'Use Code': 'use code taxroll'}

In [ ]:
fname = 'Assessor_s_Office_Secured_Tax_Roll_2018_19.csv'
data_fname = os.path.join(data_dir, fname)
df_path = process_tabular_data(data_fname, county_name,
                               col_rename_dict=column_rename_d,
                               init_output_dirs=False)

## Sales/Transfer data

In [ ]:
# due to ProgrammingError: column <column name> duplicated error thrown by create_county_redshift_table(county_name),
# had to rename these columns:
# ['use code']

column_rename_d = {'Use Code': 'use code transfer list'}

In [ ]:
fname = 'Assessor_s_Office_Ownership_Transfer_List.csv'
data_fname = os.path.join(data_dir, fname)
df_path = process_tabular_data(data_fname, county_name,
                               col_rename_dict=column_rename_d,
                               init_output_dirs=False)

# II. Combine data and add source

In [ ]:
concat_data_add_source(county_name=county_name)

### Correct duplicate columns

Correct duplicate columns in the final dataframe before posting to S3 and Redshift.

Find culprit columns, set column_rename_d, and re-run each data section as needed

In [ ]:
dup_cols = find_duplicate_cols()
dup_cols

# III. Create Redshift table

In [ ]:
create_county_redshift_table(county_name)